## Observations and Insights 

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
merged_df = pd.merge(mouse_metadata, study_results, on="Mouse ID", how = "outer")
merged_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [3]:
# Checking the number of mice in the DataFrame.
len(merged_df["Mouse ID"].unique())

249

In [56]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
temp_df = merged_df.groupby(["Mouse ID","Timepoint"]).count()
id_timepoint_df = temp_df[["Sex", "Weight (g)"]]
id_timepoint_df

Sex  Weight (g)
Mouse ID Timepoint                 
a203     0            1           1
         5            1           1
         10           1           1
         15           1           1
         20           1           1
...                 ...         ...
z969     25           1           1
         30           1           1
         35           1           1
         40           1           1
         45           1           1

[1888 rows x 2 columns]

In [4]:
# Optional: Get all the data for the duplicate mouse ID. 


In [19]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_df = merged_df.drop_duplicates("Mouse ID")
clean_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.0,0
10,s185,Capomulin,Female,3,17,0,45.0,0
20,x401,Capomulin,Female,16,15,0,45.0,0
30,m601,Capomulin,Male,22,17,0,45.0,0
40,g791,Ramicane,Male,11,16,0,45.0,0


In [9]:
# Checking the number of mice in the clean DataFrame.
len(clean_df)

249

## Summary Statistics

In [58]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for 
#each regimen

#create a temp df of the important data
temp_df = merged_df[["Drug Regimen", "Tumor Volume (mm3)"]]

#df then series of the each drugs' means
mean_df = temp_df.groupby("Drug Regimen").mean()
mean_values = mean_df["Tumor Volume (mm3)"]
#df then series of the each drugs' medians
median_df = temp_df.groupby("Drug Regimen").median()
median_values = median_df["Tumor Volume (mm3)"]
#df then series of the each drugs' standard deviations
std_df = temp_df.groupby("Drug Regimen").std()
std_values = std_df["Tumor Volume (mm3)"]
#df then series of the each drugs' SEMs
sem_df = temp_df.groupby("Drug Regimen").sem()
sem_values = sem_df["Tumor Volume (mm3)"]

#putting all series together for a summary broken down by Drug Regimen
summary_df = pd.DataFrame({
    "Mean" : mean_values,
    "Median": median_values,
    "Variance": std_values**2,
    "STD": std_values,
    "SEM": sem_values
})

#using map to format columns
summary_df["Mean"] = summary_df["Mean"].map("{:.2f}".format)
summary_df["Median"] = summary_df["Median"].map("{:.2f}".format)
summary_df["Variance"] = summary_df["Variance"].map("{:.2f}".format)
summary_df["STD"] = summary_df["STD"].map("{:.2f}".format)
summary_df["SEM"] = summary_df["SEM"].map("{:.4f}".format)
summary_df

# This method is the most straightforward, creating multiple series and putting them all together at the end.

,Mean,Median,Variance,STD,SEM
Drug Regimen,,,,,
Capomulin,40.68,41.56,24.95,4.99,0.3293
Ceftamin,52.59,51.78,39.29,6.27,0.4698
Infubinol,52.88,51.82,43.13,6.57,0.4922
Ketapril,55.24,53.70,68.55,8.28,0.6039
Naftisol,54.33,52.51,66.17,8.13,0.5965
Placebo,54.03,52.29,61.17,7.82,0.5813
Propriva,52.32,50.85,42.35,6.51,0.5129
Ramicane,40.22,40.67,23.49,4.85,0.3210
Stelasyn,54.23,52.43,59.45,7.71,0.5731


In [57]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
merged_df.groupby("Drug Regimen").describe()

# This method produces everything in a single groupby function.

Age_months                                                     \
                  count       mean       std  min    25%   50%   75%   max   
Drug Regimen                                                                 
Capomulin         230.0  13.456522  7.721423  1.0   7.00  16.5  20.0  24.0   
Ceftamin          178.0  13.247191  8.071406  2.0   6.00  12.0  20.0  24.0   
Infubinol         178.0  16.230337  7.510278  1.0   8.00  20.0  23.0  24.0   
Ketapril          188.0  15.659574  6.019670  1.0  11.75  18.0  19.0  24.0   
Naftisol          186.0  12.000000  6.715855  2.0   8.00   9.0  19.0  23.0   
Placebo           181.0  10.734807  6.354907  1.0   5.00  10.0  17.0  21.0   
Propriva          161.0  10.894410  7.304624  1.0   5.00   8.0  21.0  24.0   
Ramicane          228.0  10.684211  5.946629  1.0   7.00   9.0  18.0  23.0   
Stelasyn          181.0  12.784530  7.939562  1.0   4.00  14.0  21.0  23.0   
Zoniferol         182.0  12.598901  5.786114  2.0   8.00  12.5  16.0  24.0   

             Weight (g)             ... Tumor Volume (mm3)             \
                  count       mean  ...                75%        max   
Drug Regimen                        ...                                 
Capomulin         230.0  19.965217  ...          45.000000  48.158209   
Ceftamin          178.0  27.398876  ...          56.801438  68.923185   
Infubinol         178.0  27.196629  ...          57.314444  72.226731   
Ketapril          188.0  27.861702  ...          60.870951  78.567014   
Naftisol          186.0  27.166667  ...          59.963034  76.668817   
Placebo           181.0  27.928177  ...          59.916934  73.212939   
Propriva          161.0  27.043478  ...          56.184664  72.455421   
Ramicane          228.0  19.679825  ...          45.000000  47.622816   
Stelasyn          181.0  27.856354  ...          58.719297  75.123690   
Zoniferol         182.0  27.692308  ...          57.954259  73.324432   

             Metastatic Sites                                               
                        count      mean       std  min  25%  50%  75%  max  
Drug Regimen                                                                
Capomulin               230.0  0.713043  0.848993  0.0  0.0  0.0  1.0  3.0  
Ceftamin                178.0  1.179775  1.184283  0.0  0.0  1.0  2.0  4.0  
Infubinol               178.0  0.960674  1.027104  0.0  0.0  1.0  2.0  4.0  
Ketapril                188.0  1.297872  1.393873  0.0  0.0  1.0  2.0  4.0  
Naftisol                186.0  1.182796  1.216519  0.0  0.0  1.0  2.0  4.0  
Placebo                 181.0  1.441989  1.338824  0.0  0.0  1.0  2.0  4.0  
Propriva                161.0  0.975155  1.083688  0.0  0.0  1.0  1.0  4.0  
Ramicane                228.0  0.548246  0.691259  0.0  0.0  0.0  1.0  3.0  
Stelasyn                181.0  0.872928  0.972046  0.0  0.0  1.0  1.0  4.0  
Zoniferol               182.0  1.230769  1.248884  0.0  0.0  1.0  2.0  4.0  

[10 rows x 40 columns]

## Bar Plots

In [9]:
# Generate a bar plot showing the number of mice per time point for each treatment throughout the course of the study using pandas.
bar_df = merged_pd[["Drug Regimen", "Tumor Volume (mm3)","Timepoint"]].copy()
bar_df=bar_df.groupby(["Drug Regimen", "Timepoint"]).count()
bar_df.plot(kind="bar", figsize=(20,5))

In [10]:
# Generate a bar plot showing the number of mice per time point for each treatment throughout the course of the study using pyplot.
bar_df = merged_pd[["Drug Regimen", "Tumor Volume (mm3)","Timepoint"]].copy()
bar_df=bar_df.groupby(["Drug Regimen", "Timepoint"]).count()
x = np.arange(len(bar_df))
plt.bar(x, bar_df["Tumor Volume (mm3)"], color='r', alpha=0.5, align="edge")

## Pie Plots

In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
sex_pd = merged_pd[["Sex", "Mouse ID"]].copy()
sex_pd.head()
sex_pd= sex_pd.groupby(["Sex"]).count()
sex_pd
sex_pd.plot.pie(y='Mouse ID', figsize=(5, 5))

In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
sex_pd = merged_pd[["Sex", "Mouse ID"]].copy()
sex_pd.head()
sex_pd= sex_pd.groupby(["Sex"]).count()
sex_pd
plt.pie(sex_pd["Mouse ID"])

## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the most promising treatment regimens. Calculate the IQR and quantitatively determine if there are any potential outliers. 


In [14]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest

## Line and Scatter Plots

In [15]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin
Line_pd = merged_pd[["Drug Regimen", "Mouse ID", "Tumor Volume (mm3)"]].copy
Line_pd = pd.DataFrame(merged_pd)
Line_pd
#Line_pd.set_index('Drug Regimen')

In [16]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen



## Correlation and Regression

In [17]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
